# Succession diagram simplifications

The explanations in this notebook assume a basic understanding of stable motif succession diagrams and the PyStableMotif library by Rozum et al.:
https://github.com/jcrozum/PyStableMotifs <br>
<br>
Both kinds of succession diagrams (SD) -- the reduction based (RBSD) and the motif-based (MBSD) succession diagrams generate different pathways for different combinations of motif stabilization. The RBSD is more efficient in doing this but the end result can still be quite difficult to interpret, if the system is large and/or has a lot of motifs. This notebook aims to introduce a few simple methods to further simplify the succession diagrams and help with their interpretation. 
<br> This nodebook goes trough some of the steps of the PyStableMotif tutorial notebook, from generating the attractor repertoire to saving the succession diagrams, then introduces a few methods of simplifying these and generates two additional succession diagrams, the Simplified MBSD and the Bipartite MBSD.

In [1]:
import sys
import PyStableMotifs as sm
import PyBoolNet
import PyStableMotifs.Export as ex


Bad key "text.kerning_factor" on line 4 in
/home/david/anaconda2/envs/python_3_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Read in a Boolean model:

We use the Phase Switch model in this example from *Deritei, D., Aird, W. C., Ercsey-Ravasz, M., & Regan, E. R. (2016). Principles of dynamical modularity in biological regulatory networks. Scientific reports, 6(1), 1-19.*

In [2]:
model_name_='Phase_Switch'
rules='''Cdc20 *= pAPC and (not Cdh1) and (not Mad2)
Cdc25A *= CyclinA and (not Cdh1)
Cdc25C *= CyclinA or (CyclinB and Cdk1)
Cdh1 *= (not CyclinA) and (not (CyclinB and Cdk1))
Cdk1 *= Cdc25C and (CyclinA or CyclinB) and (Cdk1 or (not Wee1))
CyclinA *= (Cdc25A or CyclinA) and (not ((pAPC and Cdc20) or (Cdh1 and UbcH10) ) )
CyclinB *= not (pAPC and Cdc20) and (not Cdh1)
Mad2 *= not (pAPC and Cdc20) and CyclinB and Cdk1
pAPC *= (pAPC and Cdc20) or (CyclinB and Cdk1)
UbcH10 *= (not Cdh1) or (UbcH10 and (Cdc20 or CyclinA or CyclinB))
Wee1 *= not ((CyclinA or CyclinB) and Cdk1)
'''

In [3]:
#format the rules to be PyBoolNet compatible 
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
sm.Format.pretty_print_prime_rules({k:primes[k] for k in sorted(primes)})


Cdc20* = !Cdh1 & !Mad2 & pAPC
Cdc25A* = !Cdh1 & CyclinA
Cdc25C* = Cdk1 & CyclinB | CyclinA
Cdh1* = !CyclinA & !CyclinB | !Cdk1 & !CyclinA
Cdk1* = Cdc25C & CyclinB & !Wee1 | Cdc25C & CyclinA & !Wee1 | Cdc25C & Cdk1 & CyclinB | Cdc25C & Cdk1 & CyclinA
CyclinA* = CyclinA & !UbcH10 & !pAPC | !Cdh1 & CyclinA & !pAPC | Cdc25A & !UbcH10 & !pAPC | Cdc25A & !Cdh1 & !pAPC | !Cdc20 & CyclinA & !UbcH10 | !Cdc20 & !Cdh1 & CyclinA | !Cdc20 & Cdc25A & !UbcH10 | !Cdc20 & Cdc25A & !Cdh1
CyclinB* = !Cdh1 & !pAPC | !Cdc20 & !Cdh1
Mad2* = Cdk1 & CyclinB & !pAPC | !Cdc20 & Cdk1 & CyclinB
UbcH10* = CyclinB & UbcH10 | CyclinA & UbcH10 | Cdc20 & UbcH10 | !Cdh1
Wee1* = !CyclinA & !CyclinB | !Cdk1
pAPC* = Cdk1 & CyclinB | Cdc20 & pAPC


## Generating the Attractor repertoire

In [4]:
#explanation of the parameter
max_simulate_size=20

In [5]:
ar = sm.AttractorRepertoire.from_primes(primes, max_simulate_size=max_simulate_size)

### What do we know about the attractors?

If we want the attractors in a DataFrame:

In [6]:
df=ex.attractor_dataframe(ar)
df

,Cdc20,Cdc25A,Cdc25C,Cdh1,Cdk1,CyclinA,CyclinB,Mad2,UbcH10,Wee1,pAPC
0,0,0,0,1,0,0,0,0,0,1,0
1,0,0,1,0,1,0,1,1,1,0,1
2,0,1,1,0,0,1,1,0,1,1,0
3,0,1,1,0,1,1,1,1,1,0,1


In [7]:
#ar.succession_diagram.motif_reduction_dict[0].stable_motifs

## Generating and plotting the Suuccession Diagrams

If we want to add the attractors as nodes of the succession diagram connected to the terminal nodes of the succession diagram we set:


In [8]:
include_attractors_in_diagram=True

### Reduced network based succession diagram

In [9]:
GR=ex.networkx_succession_diagram_reduced_network_based(ar,include_attractors_in_diagram=include_attractors_in_diagram)

### Motif based succession diagram

In [10]:
GM=ex.networkx_succession_diagram_motif_based(ar,include_attractors_in_diagram=True)

In case we only need the set of unique stable motifs:

In [11]:
motifs_list=ar.succession_diagram.get_motifs()

motifs_dict = dict(zip(range(len(motifs_list)),motifs_list))
merged_dict=motifs_dict.copy()
attractors_dict=dict()
for a_index,a in enumerate(ar.attractors):
    merged_dict['A'+str(a_index)]=a.attractor_dict
    attractors_dict['A'+str(a_index)]=a.attractor_dict

In [12]:
GM.nodes(data=True)

NodeDataView({(0, 1): {'states': {'Cdc25A': 0, 'CyclinA': 0}, 'motif': {'Cdc25A': 0, 'CyclinA': 0}, 'label': "{'Cdc25A': 0, 'CyclinA': 0}"}, (1, 2): {'states': {'Cdh1': 1, 'CyclinB': 0, 'Cdc20': 0, 'Cdc25C': 0, 'Cdk1': 0, 'Mad2': 0, 'Wee1': 1, 'pAPC': 0, 'UbcH10': 0}, 'motif': {'Cdh1': 1, 'CyclinB': 0}, 'label': "{'Cdh1': 1, 'CyclinB': 0}"}, (1, 3): {'states': {'Cdh1': 1, 'CyclinB': 0, 'Cdc20': 0, 'Cdc25C': 0, 'Cdk1': 0, 'Mad2': 0, 'Wee1': 1, 'pAPC': 0, 'UbcH10': 0}, 'motif': {'Cdc25C': 0, 'Cdk1': 0}, 'label': "{'Cdc25C': 0, 'Cdk1': 0}"}, (1, 4): {'states': {'Cdh1': 1, 'CyclinB': 0, 'Cdc20': 0, 'Cdc25C': 0, 'Cdk1': 0, 'Mad2': 0, 'Wee1': 1, 'pAPC': 0, 'UbcH10': 0}, 'motif': {'Cdk1': 0, 'Wee1': 1}, 'label': "{'Cdk1': 0, 'Wee1': 1}"}, (1, 5): {'states': {'Cdc20': 0, 'Cdc25C': 1, 'Cdh1': 0, 'Cdk1': 1, 'CyclinB': 1, 'Mad2': 1, 'UbcH10': 1, 'Wee1': 0, 'pAPC': 1}, 'motif': {'Cdc20': 0, 'Cdc25C': 1, 'Cdh1': 0, 'Cdk1': 1, 'CyclinB': 1, 'Mad2': 1}, 'label': "{'Cdc20': 0, 'Cdc25C': 1, 'Cdh1': 0, 

## Creating the Simplified Motif Based Succession Diagram (SMBSD)

We can have a version of the MBSD where **every motif is present only once in the SD**. The possible transitions are aggregated as edges and the different possible orders and combinations are reduced to paths between motifs. The attractors are still sink-nodes, but primary motifs are no longer necessarily sources because they can stabilize also after other motifs, thus they will have incoming edges from other motifs. This creates a much simpler picture. However, the simplification comes at a cost: some primary motif -> attractor paths include nodes that are incompatible with the final attractor. 
This problem can be solved by checking the consistency of a path, using a function defined below (*has_consistent_path()*)


In [13]:
import networkx as nx
motif_keys=list(merged_dict.keys())
motif_values=list(merged_dict.values())
GMM=nx.DiGraph()
GMM.add_nodes_from(merged_dict)
for n in GMM.nodes():
    GMM.nodes[n]['states']=merged_dict[n]
for i,j in GM.edges():
    source=motif_keys[motif_values.index(GM.nodes[i]['motif'])]
    try:
        target=motif_keys[motif_values.index(GM.nodes[j]['motif'])]
    except KeyError:
        target=motif_keys[motif_values.index(GM.nodes[j]['states'])] #attractor
    GMM.add_edge(source,target)

Checking the path consistency:

In [14]:
def has_consistent_path(GMM,source,target):
    '''
    Checks if there is a consistent (no contradicting node-states) path of motifs between
    a source and a target node in a succession diagram.
    
    GMM - succession diagram DiGraph object (preferable the simplified MBSD)
    source - a node of the GMM
    target - a node of the GMM
    
    Returns: True if there is a consistent path, False otherwise
    '''
    
    
    
    for simple_path in list(nx.all_simple_paths(GMM,source,target)):
        #print (simple_path)
        path_virtual_nodes={}
        for node_id in simple_path: 
            contradiction=False
            virtual_nodes = GMM.nodes[node_id]['states']
            for virtual_node,value in virtual_nodes.items():
                #print(virtual_node, value)
                if virtual_node in path_virtual_nodes:
                    if path_virtual_nodes[virtual_node]!=value:
                        contradiction=True
                        #print('Found a contradiction')
                        break
                else:
                    path_virtual_nodes[virtual_node]=value
            if contradiction:
                break
        if contradiction==False:
            return True
    return False

## Creating the motif-attractor bipartite representation

A further simplification of the succession diagram could be if we check what motifs have a consistent path to what attractors. If we have $m$ number of motifs and $A$ number of attractors, we can create an $m$ by $A$ matrix $M$ in which $M_{ij}$ is 1 if motif $m_i$ has a consistent simple path in the SMBSD to attractor $A_j$. This is a bipartite network of motifs and attractors.

In [15]:
GM_bp=nx.DiGraph()
GM_bp.add_nodes_from(motifs_dict)
GM_bp.add_nodes_from(attractors_dict)
for m in motifs_dict:
    GM_bp.nodes[m]['states']=motifs_dict[m]
    for a in attractors_dict:
        GM_bp.nodes[a]['states']=attractors_dict[a]
        #if nx.has_path(GMM,m,a):
        if has_consistent_path(GMM,m,a):
            GM_bp.add_edge(m,a)
            
for a in attractors_dict:
    GM_bp.nodes[a]['states']=attractors_dict[a]
        
print(GM_bp.number_of_edges())

12


## Saving all 4 kinds of graphs into graphml

Plotting the succession diagrams in matplolib is an quick and efficient way of having a glimpse at the succession diagram, however in the case of large and more complex diagrams this can become inefficient. We suggest exporting the diagrams and plotting them with tools such as yED. Here we explain how to do it:
* first, we export the succession diagram into graphml format. The attributes such as the label are preserved by the format

In [16]:
ex.save_to_graphml(GR,model_name='SD_reduction_based_%s'%model_name_)
ex.save_to_graphml(GM,model_name='SD_motif_based_%s'%model_name_)
ex.save_to_graphml(GMM,model_name='SD_simplified_motif_based_%s'%model_name_)
ex.save_to_graphml(GM_bp,model_name='SD_bipartite_motif_based_%s'%model_name_)


* next, open the saved graphml in yED
* go to Edit -> Properties Mapper
* (optional) in the top left corner of the pop-up window click *import additional configuration* and select the *succession_diagram_yED_properties.cnfx* config file from GitHub. 
* Set up the configuration and click *Apply*
* Finally, we suggest a hierarchical layout. To get this go to Layout -> Hierarchical